## Phase 3 (Data Preparation): Age => AgeBinned, KnownAge, Child

* Autorin: Anna (i3-Versicherung)
* Webseite: [Data Science Training - Kapitel 5](https://data-science.training/kapitel-5/)
* Datum: 23.03.2023

Zum Attribut Alter fehlen 263 Werte. Eine geeignete Methode zum Schätzen der fehlenden Werte haben wir noch nicht gefunden. Wir können zwar ein Binning durchführen (s.u.), allerdings ist es vielleicht erst einmal besser auf dieses Attribut zu verzichten und nur die neuen Features "KnownAge" und "Child" zu benutzen.

In [4]:
# Pandas Paket (Package) importieren
#  Datenstrukturen und Datenanalyse, I/O
#  https://pandas.pydata.org/pandas-docs/stable/
import pandas as pd
# Pyplot Modul der Matplot Bibliothek (Library) importieren
#  Mathematische grafische Daten-Visualisierungen
#  https://matplotlib.org/contents.html
#import matplotlib.pyplot as plt

In [5]:
# Trainings- und Testdaten als Pandas Data Frame (df) aus CSV-Dateien laden
#  (KNIME: "CSV Reader")
df_train = pd.read_csv('../../data/titanic/original/train.csv')
df_test  = pd.read_csv('../../data/titanic/original/test.csv')

In [6]:
# Trainings- und Testdaten zusammenführen
#  (KNIME "Concatenate")
#df = pd.concat([df_train, df_test], ignore_index=True)
# Wir benutzen nur die Trainingsdaten
df = df_train

In [7]:
# Datentypen automatisch konvertieren
df = df.convert_dtypes()

In [8]:
# Fehlende Werte prüfen
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
# Spezielle Titanic-Statistik (Überlebenswahrscheinlichkeiten)
def dst_titanic_statistics(df, col):
    stats = df.groupby([col], as_index=False, observed=True)['PassengerId'].count() # Spalte "PassengerId" aus Titanic-Daten
    probs = df.groupby([col], as_index=False, observed=True)['Survived'].mean()     # Spalte "Survived"    aus Titanic-Daten
    stats = stats.merge(probs, on=col)
    stats = stats.rename(columns={'PassengerId': 'Anzahl', 'Survived': 'Überlebenswahrscheinlichkeit'})
    return stats

In [10]:
# Age Binning - Teil 1: Automatisches Binning (mittels Quantile)
df['AgeBinned'] = pd.qcut(df['Age'].dropna(), 6)
# Statistik (Überlebenswahrscheinlichkeiten)
stats = dst_titanic_statistics(df, 'AgeBinned')
display(stats)

,AgeBinned,Anzahl,Überlebenswahrscheinlichkeit
0,"(0.419, 18.0]",139,0.503597
1,"(18.0, 23.0]",107,0.308411
2,"(23.0, 28.0]",116,0.387931
3,"(28.0, 34.0]",116,0.413793
4,"(34.0, 44.0]",121,0.421488
5,"(44.0, 80.0]",115,0.373913


In [11]:
# Age Binning - Teil 2: Manuelles Binning
bins = [0, 12, 24, 45, 60, 120]
labels = ['Child', 'YoungAdult', 'Adult','OlderAdult','Senior']
df['AgeBinned'] = pd.cut(df['Age'], bins, labels=labels)
# Statistik (Überlebenswahrscheinlichkeiten)
stats = dst_titanic_statistics(df, 'AgeBinned')
display(stats)

,AgeBinned,Anzahl,Überlebenswahrscheinlichkeit
0,Child,69,0.57971
1,YoungAdult,208,0.375
2,Adult,334,0.401198
3,OlderAdult,81,0.407407
4,Senior,22,0.227273


In [12]:
# Histogramme des Attributs Age
#age = df['Age'].dropna()
#for n in range(10, 100, 10):
#    plt.hist(age, bins=n, density=True)
#    plt.title('Histogramm')
#    plt.xlabel('Alter (Age)')
#    plt.ylabel('Häufigkeit')
#    plt.show()

In [13]:
# Neues Feature "KnownAge"
#  (KNIME: "Rule Engine")
df['KnownAge'] = (df['Age'].notna()).astype('int')
# Statistik (Überlebenswahrscheinlichkeiten)
stats = dst_titanic_statistics(df, 'KnownAge')
display(stats)

,KnownAge,Anzahl,Überlebenswahrscheinlichkeit
0,0,177,0.293785
1,1,714,0.406162


In [14]:
# Neues Feature "Child"
#  (KNIME: "Rule Engine")
#
#df['Child'] = (df['Age'] < 12)          # Schritt 1: Age < 12 => True 
#df['Child'] = df['Child'].fillna(False) # Schritt 2: Missing Values => False
#df['Child'] = df['Child'].astype('int') # Schritt 3: Konvertierung Boolean => Integer
#
df['Child'] = (df['Age'] < 12).fillna(False).astype('int')
#
# Statistik (Überlebenswahrscheinlichkeiten)
stats = dst_titanic_statistics(df, 'Child')
display(stats)

,Child,Anzahl,Überlebenswahrscheinlichkeit
0,0,823,0.368165
1,1,68,0.573529


In [15]:
display(df.head(12))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeBinned,KnownAge,Child
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,<NA>,S,YoungAdult,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,<NA>,S,Adult,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S,Adult,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,<NA>,S,Adult,1,0
5,6,0,3,"Moran, Mr. James",male,<NA>,0,0,330877,8.4583,<NA>,Q,NaN,0,0
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,OlderAdult,1,0
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,<NA>,S,Child,1,1
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,<NA>,S,Adult,1,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,<NA>,C,YoungAdult,1,0


### Ergebnis

Das neue binäre Feature KnownAge führt zu Überlebenswahrscheinlichkeiten, die sich um ca. 10 Prozent unterscheiden. Beim neuen binären Feature Child sind es sogar ca. 20 Prozent. Allerdings werden nur 68 Kinder betrachtet.

Ein Binning lässt sich zwar durchführen, die Ergebnisse sind doch stark abhängig von der Art und Weise der Aufteilung der Bins. Kinder haben in jedem Fall eine höhere Überlebenswahrscheinlichkeit. Hierfür können wir aber auch das neue Feature Child benutzen. Die Überlebenswahrscheinlichkeiten zu den anderen Bins sind nicht sehr unterschiedlich und liegen meisten im Bereich von 40 Prozent. Sehr alte Passagiere haben allerdings nur eine geringe Überlebenswahrscheinlichkeit. Deren Anzahl ist jedoch auch sehr gering. Im Bereich von 18 bis 23 Jahren, also bei jungen erwachsene Passagieren, beträgt die Überlebenswahrscheinlichkeit nur knapp 31 Prozent.